In [1]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt

In [2]:
# Read csv file with results from the simulation - Bike tasks

#Import files
#os.chdir("/Users/naroacorettisanchez/Documents/GitHub/MultifunctionalVehicleFleets/results/ MIXED_SS_fixedfleet")

os.chdir("/Users/naroacorettisanchez/Documents/GitHub/MultifunctionalVehicleFleets/results/38reps_MIXED_fullData_final")
extension = 'csv'

#Bike tasks
filenames =[i for i in glob.glob('autonomousBike_trip_event*.{}'.format(extension))]

#Create a list with the dataframes in filenames
dfs = []
for filename in filenames:
    df_tasks = pd.read_csv(filename)
    dfs.append(df_tasks)


In [3]:
# Create a new list to store the modified dataframes
dfs_all= []

# Iterate over each dataframe in dfs
for df_tasks in dfs:
    # Select the desired columns
    df_modified = df_tasks[['Cycle', 'Day', 'Time', 'Agent', 'Message', 'Distance Traveled']]
    # Append the modified dataframe to the new list
    dfs_all.append(df_modified)


dfs_all[0].head()

,Cycle,Day,Time,Agent,Message,Distance Traveled
0,0,7,00:00:00,autonomousBike0,nil,NaN
1,0,7,00:00:00,autonomousBike1,nil,NaN
2,0,7,00:00:00,autonomousBike2,nil,NaN
3,0,7,00:00:00,autonomousBike3,nil,NaN
4,0,7,00:00:00,autonomousBike4,nil,NaN


In [4]:
# Create a new list to store copies of the original DataFrames
dfs_modified = []

# Use a loop to create copies of each DataFrame in the list
for df in dfs_all:
    dfs_modified.append(df.copy())

In [5]:
# for df in dfs_modified: Keep only the rows with 'Message' starting by 'In Use*'
for df in dfs_modified:

    ##CHANGE FOR COUNTING TRANSITIONS
    #df.drop(df[~df['Message'].str.startswith('In Use')].index, inplace = True)
    df.drop(df[~df['Message'].str.startswith('Picked up')].index, inplace = True)

dfs_modified[0].head()

,Cycle,Day,Time,Agent,Message,Distance Traveled
341,769,7,01:04:05,autonomousBike293,Picked up people(3471),273.317404
343,773,7,01:04:25,autonomousBike148,Picked up people(3472),390.925521
347,835,7,01:09:35,autonomousBike319,Picked up people(3473),119.020839
353,899,7,01:14:55,autonomousBike166,Picked up package(0),147.992348
357,918,7,01:16:30,autonomousBike4,Picked up people(3474),0.000000


In [6]:
# Crop Message column string to get only the task name

for df in dfs_modified:
    df['Message'] = df['Message'].str.split('(').str[0]

dfs_modified[0].head()


,Cycle,Day,Time,Agent,Message,Distance Traveled
341,769,7,01:04:05,autonomousBike293,Picked up people,273.317404
343,773,7,01:04:25,autonomousBike148,Picked up people,390.925521
347,835,7,01:09:35,autonomousBike319,Picked up people,119.020839
353,899,7,01:14:55,autonomousBike166,Picked up package,147.992348
357,918,7,01:16:30,autonomousBike4,Picked up people,0.000000


In [7]:
# Create a new list to store the modified dataframes
dfs_prev_message = []

for df_tasks_use in dfs_modified:
    # Reorder rows to get all the events for each agent together
    df_tasks_use = df_tasks_use.sort_values(by=['Agent', 'Day', 'Time'])
    #Add a column with the previous message, in case it is from the same agent
    df_tasks_use['Previous_Message'] = df_tasks_use['Message'].shift().where(df_tasks_use['Agent'].eq(df_tasks_use['Agent'].shift()), np.nan)
    dfs_prev_message.append(df_tasks_use)

dfs_prev_message[0].head()

,Cycle,Day,Time,Agent,Message,Distance Traveled,Previous_Message
3281,6122,7,08:30:10,autonomousBike0,Picked up package,217.071669,NaN
5075,6564,7,09:07:00,autonomousBike0,Picked up people,1209.529682,Picked up package
6618,7054,7,09:47:50,autonomousBike0,Picked up people,66.406191,Picked up people
7189,7198,7,09:59:50,autonomousBike0,Picked up package,193.652503,Picked up people
8598,7591,7,10:32:35,autonomousBike0,Picked up package,537.552300,Picked up package


In [8]:
for df_tasks_use in dfs_prev_message:
    # For each row, check if Message is the same as Previous_Message, if yes add True to the column 'Same'
    df_tasks_use['Same'] = np.where(df_tasks_use['Message'] == df_tasks_use['Previous_Message'], True, False)
    # For each row, if Previous Message is NaN, add NaN to the column 'Same'
    df_tasks_use['Same'] = np.where(df_tasks_use['Previous_Message'].isnull(), np.nan, df_tasks_use['Same'])
    
dfs_prev_message[0].head()

,Cycle,Day,Time,Agent,Message,Distance Traveled,Previous_Message,Same
3281,6122,7,08:30:10,autonomousBike0,Picked up package,217.071669,NaN,NaN
5075,6564,7,09:07:00,autonomousBike0,Picked up people,1209.529682,Picked up package,0.0
6618,7054,7,09:47:50,autonomousBike0,Picked up people,66.406191,Picked up people,1.0
7189,7198,7,09:59:50,autonomousBike0,Picked up package,193.652503,Picked up people,0.0
8598,7591,7,10:32:35,autonomousBike0,Picked up package,537.552300,Picked up package,1.0


In [9]:

average_counts = []
for df_tasks_use in dfs_prev_message:
    counts = df_tasks_use['Same'].value_counts(normalize=True) * 100
    average_counts.append(counts)

average_values = pd.concat(average_counts).groupby(level=0).mean()
print(average_values)

# O are the tasks switches


0.0    36.026476
1.0    63.973524
Name: Same, dtype: float64


In [10]:
total_distance = 0

for df_tasks_use in dfs_all:
    total_distance += df_tasks_use['Distance Traveled'].sum()

print(total_distance)


3409556833.613477


In [11]:
distance_task_change = 0

for df_tasks_use in dfs_prev_message:
    distance_task_change += df_tasks_use.loc[df_tasks_use['Same'] == 0, 'Distance Traveled'].sum()

print(distance_task_change)

print('Distance traveled in task change pickup: ', distance_task_change/total_distance*100, '%')

296282768.5163519
Distance traveled in task change pickup:  8.689773568090047 %


In [12]:
distance_same_task = 0

for df_tasks_use in dfs_prev_message:
    distance_same_task  += df_tasks_use.loc[df_tasks_use['Same'] == 1, 'Distance Traveled'].sum()

print(distance_same_task)

print('Distance traveled same task pickup: ', distance_same_task /total_distance*100, '%')

534483023.74727327
Distance traveled same task pickup:  15.676026235375101 %
